In [204]:
import requests
import pandas as pd
from datetime import datetime

In [205]:
target_account = "TKqUVHFV4ayYZR138m2BiVPktn9XZXLZ7W"


In [206]:

df = pd.DataFrame(columns=['time','tx_hash','tx_type',
'coin','token_type','amount','from_address','to_address'])

In [207]:
url = f"https://apilist.tronscan.org/api/transaction?sort=-timestamp&count=true&limit=20&start=0&address={target_account}"
res = requests.get(url)
json_data = res.json()

In [208]:
total_num = json_data['total']
txs = json_data['data']


In [209]:
number_of_iter = ( total_num // 20 ) + 1
number_of_iter

5

In [210]:

for tx in txs:
    tx_time_stamp = datetime.fromtimestamp(tx['timestamp']/1000)
    tx_hash = tx['hash']
    tx_result = tx['result']
    if tx_result != 'SUCCESS':
        continue
    tx_coin = tx['tokenInfo']['tokenName']
    tx_token_decimal = tx['tokenInfo']['tokenDecimal']
    tx_amount = int(tx['amount']) / tx_token_decimal
    tx_from_address = tx['ownerAddress']
    tx_to_address = tx['toAddress']
    tx_token_type = tx['tokenType']
    try:
        tx_type = tx['trigger_info']['methodName']
    except:
        tx_type = "Transfer TRX"
    df = pd.concat([df, pd.DataFrame([[tx_time_stamp, tx_hash,tx_type, 
    tx_coin, tx_token_type,tx_amount,tx_from_address ,
    tx_to_address]], 
    columns=['time','tx_hash','tx_type',
'coin','token_type','amount','from_address','to_address'])])
    



In [212]:
for i in range(1, number_of_iter):

    start = 20 * i
    
    url = f"https://apilist.tronscan.org/api/transaction?sort=-timestamp&count=true&limit=20&start={start}&address={target_account}"
    res = requests.get(url)
    json_data = res.json()
    # tx를 안받아왔다
    txs = json_data['data']
    for tx in txs:
        tx_time_stamp = datetime.fromtimestamp(tx['timestamp']/1000)
        tx_hash = tx['hash']
        tx_result = tx['result']
        if tx_result != 'SUCCESS':
            continue
        tx_coin = tx['tokenInfo']['tokenName']
        tx_token_decimal = tx['tokenInfo']['tokenDecimal']
        tx_amount = int(tx['amount']) / tx_token_decimal
        tx_from_address = tx['ownerAddress']
        tx_to_address = tx['toAddress']
        tx_token_type = tx['tokenType']
        try:
            tx_type = tx['trigger_info']['methodName']
        except:
            tx_type = "Transfer TRX"
        # df를 합치는 데에서 오류가 있는 거 같다.
        df = pd.concat([df, pd.DataFrame([[tx_time_stamp, tx_hash,tx_type, 
        tx_coin, tx_token_type,tx_amount,tx_from_address ,
        tx_to_address]], 
        columns=['time','tx_hash','tx_type',
    'coin','token_type','amount','from_address','to_address'])])
    # 뭐가 틀렸을까...?    


    


https://apilist.tronscan.org/api/transaction?sort=-timestamp&count=true&limit=20&start=20&address=TKqUVHFV4ayYZR138m2BiVPktn9XZXLZ7W
https://apilist.tronscan.org/api/transaction?sort=-timestamp&count=true&limit=20&start=40&address=TKqUVHFV4ayYZR138m2BiVPktn9XZXLZ7W
https://apilist.tronscan.org/api/transaction?sort=-timestamp&count=true&limit=20&start=60&address=TKqUVHFV4ayYZR138m2BiVPktn9XZXLZ7W
https://apilist.tronscan.org/api/transaction?sort=-timestamp&count=true&limit=20&start=80&address=TKqUVHFV4ayYZR138m2BiVPktn9XZXLZ7W


In [216]:
file_name =  F'트론 입출금 {target_account}.xlsx'
df.to_excel(file_name, index=False)
